# Python Jarvis

## Purpose
We wanted to create a voice activated assistant coded in Python. The final idea was for it to replicate the functionality of popular voice assistants such as Alexa, Siri, and Google Voice Assistant. We believed that such a program would be helpful to do simple computer tasks for those with disabilities or other issues.

## GitHub Repository Link
https://github.com/ironspur5/PythonVoiceAssistant

## Requirements
**Environment:** 

MacOS 10.14.13

Anaconda 

Python 3.6

**Steps:**

1.Create Google Cloud Platform account and activate a cloud speech to text API .JSON key

2.Create Python 3.6 environment in Anaconda

3.Activate Python 3.6 environment and verify it is active with * icon next to it

    conda activate py36
    conda env list

4.Install dependencies with following pip command (change path/to/ accordingly)

    pip install -r /path/to/requirements.txt 

5.Mac only: Install Homebrew with following command

    /usr/bin/ruby -e "$(curl -fsSL https://raw.githubusercontent.com/Homebrew/install/master/install)"

6.Install Portaudio with following command
 
    brew install portaudio 

7.Install mpg123 

    brew install mpg123

## Process
We utilized available Python libraries to handle processing speech commands and outputting audio responses. We imported the following libraries:

In [ ]:
import speech_recognition as sr
from datetime import datetime
import time
import os
from gtts import gTTS
import webbrowser
import random
from tkinter import *

After prompting the user for input, the program will continue to run and listen for commands.

In [ ]:
speak("Hi, what can I do for you?")
while 1:
    data = recordAudio()
    jarvis(data)

__recordAudio__
Uses the system microphone as its source and then records the audio it collects.

In [ ]:
r = sr.Recognizer()
with sr.Microphone() as source:
    audio = r.listen(source)

Using Google's speech recognition, we then turn the collected audio data into a string or print an error message if the text cannot be processed.

In [ ]:
try:
    # Uses the default API key
    # To use another API key: `r.recognize_google(audio, key="GOOGLE_SPEECH_RECOGNITION_API_KEY")`
    data = r.recognize_google(audio)
    print("You said: " + data)
except sr.UnknownValueError:
    print("Google Speech Recognition could not understand audio")
except sr.RequestError as e:
    print("Could not request results from Google Speech Recognition service; {0}".format(e))

return data

__jarvis__
The data string is then passed to jarvis which handles the command.

In [ ]:
def jarvis(data):
    if "how are you" in data:
        speak("I am fine")
        #LABEL = Label(ROOT, text="I am fine")
        #LABEL.pack()

    # previously it was if "joke" or "jokes" in data
    # thus caused the joke case to run every time! Deleting or "jokes" fixed the issue
    if "joke" in data:
        # get a random number between 0 and 75 to get a random joke
        ranNum = random.randint(0, 75)
        # preload joke before request
        jokeDict = eval(open("dict.txt").read())
        setUpline = jokeDict[ranNum]["setup"]
        punchy = jokeDict[ranNum]["punchline"]
        speak(setUpline)
        speak(punchy)

    if "time" in data:
        tm = datetime.now()
        speak("The time is " + tm.strftime("%I:%M%p"))
        #LABEL = Label(ROOT, text=tm.strftime("%I:%M%p"))
        #LABEL.pack()

    if "date" in data:
        now = datetime.now()
        speak("Today is " + now.strftime("%A,%d %B,%Y"))
        #LABEL = Label(ROOT, text=now.strftime("%A,%d %B,%Y"))
        #LABEL.pack()

    if "news" in data:
        speak("Coming right up")
        #LABEL = Label(ROOT, text="Coming right up")
        #LABEL.pack()
        webbrowser.open('https://news.google.com')
    
    if "weather" in data:
        speak(str(getWeather()))

__speak__
This function uses Google Text to Speech to convert a string response into an MP3 audio file. It then uses mpg123 which has been previously installed to play the audio clip. 

In [ ]:
def speak(audioString):
    print(audioString)
    tts = gTTS(text=audioString, lang='en')
    tts.save("audio.mp3")
    os.system("mpg123 audio.mp3")

__weather__
Utilizes a web scraper to collect hourly data from weather.com and then outputs the current weather conditions.

In [ ]:
def getWeather():
    page = requests.get("https://weather.com/weather/hourbyhour/l/7472a7bbd3a7454aadf596f0ba7dc8b08987b1f7581fae69d8817dffffc487c2") #https://weather.com/weather/tenday/l/7472a7bbd3a7454aadf596f0ba7dc8b08987b1f7581fae69d8817dffffc487c2
    content = page.content
    soup = BeautifulSoup(content, "html.parser")
    l = []
    all = soup.find("div", {"class": "locations-title hourly-page-title"}).find("h1").text #"locations-title ten-day-page-title"
    #print(all)

    table = soup.find_all("table", {"class": "twc-table"})
    #print(table)
    for items in table:
        for i in range(len(items.find_all("tr")) - 1):
            d = {}
            try:
                d["desc"] = items.find_all("td", {"class": "description"})[i].text
                d["temp"] = items.find_all("td", {"class": "temp"})[i].text
            except:
                d["desc"] = "None"
                d["temp"] = "None"
            l.append(d)
            
            import pandas

            df = pandas.DataFrame(l)
            #print(df.temp.loc[[0]] + " " + df.desc.loc[[0]])

            response = "It is " + df.at[0, "desc"] + ". The temperature is " + df.at[0, "temp"]

            print(response)

            return str(response)

The string is returned to jarvis which then uses speak to tell the user the weather.